In [19]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator 
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageOps
import requests
import torch
import json

In [20]:
model = YOLO("../YOLO/detect/train2/weights/best.pt")

In [21]:
imgs = os.listdir("./dataset/yes_exif_tag")

In [22]:
# Input: [[x1, y1, x2, y2], [x1, y1, x2, y2],...]
# output: [min_x1, min_y1, max_x2, max_y2]
def get_xy(xy_box, max_x, max_y):
    tmp = results[0].boxes.xyxy
    x_tmp = torch.cat((tmp[:, 0], tmp[:,2]))
    y_tmp = torch.cat((tmp[:, 1], tmp[:,3]))

    x1, y1, x2, y2 = min(x_tmp).item(), min(y_tmp).item(), max(x_tmp).item(), max(y_tmp).item()
    
    # avg_x, avg_y = (x1+x2)/2, (y1+y2)/2
    # half_x, half_y = avg_x-x1, avg_y-y1

    # mul = 1.0
    # x1 = max(0, avg_x-mul*half_x)
    # y1 = max(0, avg_y-mul*half_y)
    # x2 = min(max_x, avg_x+mul*half_x)
    # y2 = min(max_y, avg_y+mul*half_y)
    
    return x1, y1, x2, y2

In [23]:
def remove_rotation_exif(image_path, new_image_path):
    try:
        # 이미지를 열기
        img = Image.open(image_path)

        # RGBA에서 RGB로 변환
        if img.mode == 'RGBA':
            img = img.convert('RGB')

        # 회전 정보 적용
        img_corrected = ImageOps.exif_transpose(img)

        # 회전 정보가 적용된 이미지를 새로운 파일로 저장
        img_corrected.save(new_image_path, format='JPEG')
        print(f"회전 정보 제거 후 이미지가 저장되었습니다: {new_image_path}")

    except Exception as e:
        print(f"이미지 처리 중 오류 발생: {e}")

In [24]:
API_PATH = "http://localhost:8000/upload_file"
not_processed = []

for img_name in imgs:
    if not img_name.endswith(".JPG"):
        continue
    img_path = f"./dataset/yes_exif_tag/{img_name}"
    new_img_path = f"./dataset/no_exif_tag/{img_name}"
    remove_rotation_exif(img_path, new_img_path)
    
    results = model.predict([new_img_path])
    xy_box = results[0].boxes.xyxy
    if len(xy_box) == 0:
        not_processed.append(img_name)
        continue

    shape = cv2.imread(new_img_path).shape
    x1, y1, x2, y2 = get_xy(xy_box, shape[0], shape[1])
    data = json.dumps({"x1": x1, "y1": y1, "x2": x2, "y2": y2})  # 좌표 데이터를 JSON 형식으로 변환

    
    with open(new_img_path, "rb") as img_file:
        files = {"files": (img_name, img_file, "image/jpeg")}  # 업로드할 이미지 파일 설정
        response = requests.post(API_PATH, files=files, data={"coordinates": data})  # 파일과 데이터를 함께 POST 요청

    print(response.json())  # 서버 응답 출력

회전 정보 제거 후 이미지가 저장되었습니다: ./dataset/no_exif_tag/00009769.JPG

0: 640x480 2 화상s, 22.2ms
Speed: 3.4ms preprocess, 22.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)
{'message': '파일 업로드 완료'}
회전 정보 제거 후 이미지가 저장되었습니다: ./dataset/no_exif_tag/00007460.JPG

0: 480x640 1 화상, 28.7ms
Speed: 1.5ms preprocess, 28.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
{'message': '파일 업로드 완료'}
회전 정보 제거 후 이미지가 저장되었습니다: ./dataset/no_exif_tag/00008838.JPG

0: 480x640 1 화상, 23.1ms
Speed: 1.6ms preprocess, 23.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
{'message': '파일 업로드 완료'}
회전 정보 제거 후 이미지가 저장되었습니다: ./dataset/no_exif_tag/00007448.JPG

0: 480x640 1 화상, 19.4ms
Speed: 1.6ms preprocess, 19.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
{'message': '파일 업로드 완료'}
회전 정보 제거 후 이미지가 저장되었습니다: ./dataset/no_exif_tag/00009269.JPG

0: 480x640 1 화상, 19.2ms
Speed: 1.7ms preprocess, 19.2ms inference, 0.9ms postprocess per image at shape (1, 3,

In [25]:
not_processed

['00009821.JPG',
 '00007285.JPG',
 '00009621.JPG',
 '00008882.JPG',
 '00007428.JPG',
 '00008376.JPG',
 '00009158.JPG',
 '00008083.JPG',
 '00008101.JPG',
 '00009645.JPG',
 '00009392.JPG',
 '00008314.JPG',
 '00009394.JPG',
 '00007559.JPG',
 '00009434.JPG',
 '00007950.JPG',
 '00008360.JPG',
 '00008908.JPG',
 '00007289.JPG',
 '00009356.JPG',
 '00008415.JPG',
 '00008297.JPG',
 '00009320.JPG',
 '00008909.JPG',
 '00008663.JPG',
 '00007098.JPG',
 '00008364.JPG',
 '00007319.JPG',
 '00009614.JPG',
 '00008557.JPG',
 '00009576.JPG',
 '00007600.JPG',
 '00008734.JPG',
 '00008692.JPG',
 '00007231.JPG',
 '00008689.JPG',
 '00009688.JPG',
 '00007951.JPG',
 '00008191.JPG',
 '00007054.JPG',
 '00007582.JPG',
 '00008721.JPG',
 '00009857.JPG',
 '00009799.JPG',
 '00008291.JPG',
 '00009414.JPG',
 '00008178.JPG',
 '00008549.JPG',
 '00008450.JPG',
 '00009575.JPG',
 '00009935.JPG',
 '00009541.JPG',
 '00009330.JPG',
 '00007927.JPG',
 '00008293.JPG',
 '00007230.JPG',
 '00009663.JPG',
 '00007639.JPG',
 '00008135.JPG

# Test

In [7]:
# img_name = "00009962.JPG"
# img_path = f"./dataset/{img_name}"
# results = model.predict([img_path])



0: 480x640 1 화상, 21.1ms
Speed: 20.4ms preprocess, 21.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)


In [8]:
# xy_box = results[0].boxes.xyxy
# if len(xy_box) == 0:
#     not_processed.append(img_name)

In [9]:
# xy_box

tensor([[368.7955, 303.1767, 639.8256, 452.0403]], device='cuda:0')

In [49]:
# shape = cv2.imread(img_path).shape
# x1, y1, x2, y2 = get_xy(xy_box, shape[0], shape[1])
# data = json.dumps({"x1": x1, "y1": y1, "x2": x2, "y2": y2})  # 좌표 데이터를 JSON 형식으로 변환


In [51]:
# shape

(1024, 768, 3)

In [50]:
# with open(img_path, "rb") as img_file:
#     files = {"files": (img_name, img_file, "image/jpeg")}  # 업로드할 이미지 파일 설정
#     response = requests.post(API_PATH, files=files, data={"coordinates": data})  # 파일과 데이터를 함께 POST 요청

# print(response.json())  # 서버 응답 출력

{'message': '파일 업로드 완료'}
